In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

### Simple Random Sampling with Probabilities

In [2]:
# Global Variable
user_samples = 70000  # Number of users in test 
num_groups = 5  # Number of groups
seed = 0  
num_weeks = 4  # Number of weeks (maximum duration of test)
num_products = 2  # Number of notifications pushed per week
# Set up a dictionary contains probabilities for each group around baseline conversion rate(0.02)
prob_dict = {} 
prob_dict['a1'] = [0.02, 0.05, 0.1]  # Conversion rate of group A1 by remote, checkout, general respectively
prob_dict['a2'] = [0.025, 0.05, 0.1]
prob_dict['a3'] = [0.03, 0.055, 0.12]
prob_dict['a4'] = [0.02, 0.05, 0.1]
prob_dict['a5'] = [0.019, 0.05, 0.1]

In [3]:
prob_df = pd.DataFrame(prob_dict)

In [4]:
prob_df['metrics'] = ['Remote','Checkout','General']

In [5]:
print('Probabilities DataFrame')
print('')
print(prob_df.set_index('metrics'))

Probabilities DataFrame

            a1     a2     a3    a4     a5
metrics                                  
Remote    0.02  0.025  0.030  0.02  0.019
Checkout  0.05  0.050  0.055  0.05  0.050
General   0.10  0.100  0.120  0.10  0.100


In [6]:
# Create DataFrame
df = pd.DataFrame(columns=['user_id'],data=np.arange(1,user_samples+1))

In [7]:
def assign_group(x):
    """Assign group to each user id"""
    for i in range(1,num_groups+1):
        if x>=i and x<=(user_samples/num_groups)*i:
            return 'a' + str(i)

In [8]:
df['group'] = df['user_id'].apply(assign_group)

In [9]:
# Set up Algorithm by group
df['alg'] = np.where(df['group'].isin(['a1','a2','a4']),1,2)

In [10]:
# Set up Threshold by group
df['threshold'] = np.where(df['group'].isin(['a1','a2','a3']),0.5,0.76)

In [11]:
# Set up Notifications by group
df['noti'] = np.where(df['group'].isin(['a1']),'No','Yes')

In [12]:
# Random sample user type
df['type'] = np.random.choice(['New','Last3','Active'],size=(len(df),), p=[0.6, 0.2, 0.2])

In [13]:
# Extend dataframe by number of weeks and products/notifications
df_mp = df.append([df]*(num_products*num_weeks-1), ignore_index=True)

In [14]:
df_mp = df_mp.sort_values(by='user_id').reset_index(drop=True)

In [15]:
df_mp['product_no'] = [i+1 for i in range(num_products)]*num_weeks*user_samples

In [16]:
df_mp['week'] = sorted([i+1 for i in range(num_weeks)]*2)*user_samples

In [17]:
# Random sample results of remote/checkout/general by probabilities dataframe above
remote = []
checkout = [] 
general = []
for i in df_mp.group.unique():
    remote_ = np.random.choice([0, 1], size=(int(len(df_mp)/num_groups),), p=[1-prob_dict[i][0], prob_dict[i][0]])
    remote = np.append(remote,remote_)
    checkout_ = np.random.choice([0, 1], size=(int(len(df_mp)/num_groups),), p=[1-prob_dict[i][1], prob_dict[i][1]])
    checkout = np.append(checkout,checkout_)
    general_ = np.random.choice([0, 1], size=(int(len(df_mp)/num_groups),), p=[1-prob_dict[i][2], prob_dict[i][2]])
    general = np.append(general,general_)

In [18]:
df_mp['remote'] = remote
df_mp['checkout'] = checkout
df_mp['general'] = general
df_mp['general'] = np.where((df_mp['remote']==1)|(df_mp['checkout']==1),1,df_mp['general'])

In [19]:
# Final table for AB Test Conversion of 2 recommended products
df_mp.head(10)

,user_id,group,alg,threshold,noti,type,product_no,week,remote,checkout,general
0,1,a1,1,0.5,No,New,1,1,1.0,0.0,1.0
1,1,a1,1,0.5,No,New,2,1,0.0,0.0,0.0
2,1,a1,1,0.5,No,New,1,2,0.0,0.0,0.0
3,1,a1,1,0.5,No,New,2,2,1.0,0.0,1.0
4,1,a1,1,0.5,No,New,1,3,0.0,0.0,0.0
5,1,a1,1,0.5,No,New,2,3,0.0,0.0,0.0
6,1,a1,1,0.5,No,New,1,4,0.0,1.0,1.0
7,1,a1,1,0.5,No,New,2,4,0.0,0.0,0.0
8,2,a1,1,0.5,No,New,1,1,0.0,0.0,0.0
9,2,a1,1,0.5,No,New,2,1,0.0,0.0,0.0


In [20]:
# Sampling sales
df['sales_remote'] = np.random.randint(10, size = len(df))
df['sales_checkout'] = np.random.randint(20, size = len(df))
df['sales_general'] = np.random.randint(100, size = len(df))

In [21]:
# Sampling Scan and Go usage
df['sng_use'] = np.random.choice([0, 1], size=(int(len(df),)), p=[0.4, 0.6])

In [22]:
# Final table for AB Test Conversion of New Users and Total sales
df.head(5)

,user_id,group,alg,threshold,noti,type,sales_remote,sales_checkout,sales_general,sng_use
0,1,a1,1,0.5,No,New,4,19,24,0
1,2,a1,1,0.5,No,New,6,0,53,1
2,3,a1,1,0.5,No,New,5,0,53,1
3,4,a1,1,0.5,No,New,1,4,10,1
4,5,a1,1,0.5,No,Last3,4,5,34,1


### Permutation Testing

The reason why I chose permutation test is on README file.  

**Test:** 
- H0: Conversion rate of treatment = conversion rate of controlled
- H1: Conversion rate of treatment > conversion rate of controlled

**Permutation test guideline:**  
**0.** Calculate difference of conversion rates of control and treatment group, regard as observation  
**1.** Concatenate control and treatment group into 1 big group, act like two groups are one under null hypothesis  
**2.** Shuffle and pick a new control group and a new treatment group 
**3.** Calculate difference of conversion rates  
**4.** Repeat step 2 and 3 10000 times   
**5.** Under central limit theorem, now we have a normal distribution of conversion rate/mean difference with a sample size of 10000  
**6.** Count number of times permutation replicates (conversion rate difference) that is bigger than observation over 10000 replicates, the fraction is p-value   
**7.** If p-value < 0.05 (standard alpha), conclude rejecting null hypothesis, else accept  

In [23]:
def permutation_sample(ctrl, tm):
    """Resample control and treatment group"""
    data = np.concatenate((ctrl,tm))
    permuted_data = np.random.permutation(data)
    perm_sample_1 = permuted_data[:len(ctrl)]
    perm_sample_2 = permuted_data[len(ctrl):]
    return perm_sample_1, perm_sample_2

In [24]:
def diff_frac(ctrl, tm):
    """Calculate difference of conversion rates"""
    frac_c = np.sum(ctrl)/len(ctrl)
    frac_t = np.sum(tm)/len(tm)
    return frac_t - frac_c

In [25]:
def diff_mean(ctrl, tm):
    """Calculate difference of means"""
    mean_c = np.mean(ctrl)
    mean_t = np.mean(tm) 
    return mean_t - mean_c

In [26]:
def draw_perm_reps(ctrl, tm, size, func):
    """Resample 10000 times, calculate difference and record in an array"""
    perm_replicates = np.empty(size)
    for i in tqdm(range(size)):
        perm_sample_1, perm_sample_2 = permutation_sample(ctrl, tm)
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates

In [27]:
def test_result(ctrl, tm, size, func):
    """Caculate p value and return test results"""
    dict_ = {}
    perm_replicates = draw_perm_reps(ctrl, tm, size, func)
    conversion_t = np.sum(tm)
    sample_t = len(tm)
    obs = func(ctrl, tm)
    change = func(ctrl, tm)/(np.sum(ctrl)/len(ctrl))
    p_value = float(np.sum(perm_replicates>=obs)/len(perm_replicates))
    dict_['Conversions'] = str(int(conversion_t))+'/'+str(sample_t)
    dict_['Difference'] = str(round(obs,4))
    dict_['Change Variation'] = str(round(change*100,2)) + '%'
    dict_['P Value'] = round(p_value,2)
    return dict_

In [28]:
def all_results(df, target, size, func):
    """This function is to avoid repetitions, return test result for every group"""
    c = df[(df['group']=='a1')&(df['week']==1)][target].values
    list_results = []
    for i in df.group.unique():
        t = df[(df['group']==i)&(df['week']==1)][target].values
        t_result = test_result(c,t,size,func)
        list_results.append(t_result)
    dframe = pd.DataFrame(list_results)
    dframe['Name'] = ['Controlled','Treatment 1','Treatment 2','Treatment 3','Treatment 4']
    dframe = dframe.set_index('Name')
    return dframe

In [29]:
def all_results_2(df, target, size, func):
    """Like all_results function but modified for different dataframe"""
    c = df[(df['group']=='a1')][target].values
    list_results = []
    for i in df.group.unique():
        t = df[(df['group']==i)][target].values
        t_result = test_result(c,t,size,func)
        if func == diff_mean:
            del t_result['Conversions']
        list_results.append(t_result)
    dframe = pd.DataFrame(list_results)
    dframe['Name'] = ['Controlled','Treatment 1','Treatment 2','Treatment 3','Treatment 4']
    dframe = dframe.set_index('Name')
    return dframe    

In [ ]:
df_rm = all_results(df_mp, 'remote', 10000, diff_frac)
df_ck = all_results(df_mp, 'checkout', 10000, diff_frac)
df_ge = all_results(df_mp, 'general', 10000, diff_frac)

 91%|█████████ | 9111/10000 [00:08<00:00, 1120.28it/s]

In [ ]:
print('REMOTE SHOPPING')
print('')
print(df_rm)
print('')
print('=============================================================')
print('')
print('FAST CHECKOUT')
print('')
print(df_ck)
print('')
print('=============================================================')
print('')
print('GENERAL')
print('')
print(df_ge)

In [ ]:
#T-test Coding, for Document only

In [ ]:
# import math
# from scipy import stats

In [ ]:
# SE_c = math.sqrt(frac_c*(1-frac_c)/len(c))

In [ ]:
# SE_t = math.sqrt(frac_t*(1-frac_t)/len(t))

In [ ]:
# z = (frac_t - frac_c)/math.sqrt(SE_c**2 + SE_t**2)

In [ ]:
# pvalue = stats.t.sf(z,len(c) -2)

In [ ]:
# print('P value: ' + str(pvalue))

In [ ]:
# uc = diff_AB + critical_value * SE
# lc = diff_AB - critical_value * SE
# critical_value = stats.t.ppf(0.95, n_A -1)

In [ ]:
#Sample size

In [ ]:
# from statsmodels.stats import power as pwr
# import statsmodels.api as sm

In [ ]:
# effect = 0.2
# power = 1
# alpha = 0.05
# ratio = 1
# analysis = pwr.TTestIndPower()
# ssresult = analysis.solve_power(effect_size=effect, power=power, alpha=alpha, nobs1=None, ratio=ratio)
# print(ssresult)

In [ ]:
df.head()

In [ ]:
df_rm_sales = all_results_2(df, 'sales_remote', 10000, diff_mean)
df_ck_sales = all_results_2(df, 'sales_checkout', 10000, diff_mean)
df_ge_sales = all_results_2(df, 'sales_general', 10000, diff_mean)

In [ ]:
print('REMOTE SHOPPING SALES')
print('')
print(df_rm_sales)
print('')
print('=============================================================')
print('')
print('FAST CHECKOUT SALES')
print('')
print(df_ck_sales)
print('')
print('=============================================================')
print('')
print('GENERAL SALES')
print('')
print(df_ge_sales)

In [ ]:
# Set up probabilities dictionary for new user conversions
prob_new = {} 
prob_new['a1'] = 0.02
prob_new['a2'] = 0.018
prob_new['a3'] = 0.025
prob_new['a4'] = 0.021
prob_new['a5'] = 0.019

In [ ]:
df_new = df[df['type']=='New'].reset_index(drop=True)

In [ ]:
# Sample by probabilities dictionary above
sng = []
for i in df_new.group.unique():
    sng_ = np.random.choice([0, 1], size=(int(len(df_new)/num_groups),), p=[1-prob_new[i], prob_new[i]])
    sng = np.append(sng,sng_)

In [ ]:
result_sng = np.zeros(len(df_new))

In [ ]:
result_sng[:42125] = sng

In [ ]:
df_new['sng_use'] = result_sng

In [ ]:
df_new_result = all_results_2(df_new, 'sng_use', 10000, diff_frac)

In [ ]:
df_new_result

### Multivariate Test

After choosing the winner from multiple A/B Test, purpose of multivariate test is to figure out which factor make a winner.  
ChiSquare test is chosen since our metrics are categorical (converted/not converted)  
**H0: 2 categorical variable are independent**  
**H1: 2 categorical variable are dependent**  
This test is for 4 treatment groups with 2x2 factors

In [ ]:
# ANOVA test for continuous variable only
# from statsmodels.stats import power as pwr
# import statsmodels.api as sm

# model = sm.formula.ols('remote ~ alg + threshold + alg:threshold', data = df_mp).fit()
# aov_table = sm.stats.anova_lm(model, typ=2)
# print(aov_table)

In [ ]:
from scipy.stats import chisquare
def chisquaretest(df, factor, target, alpha):
    """Return chi-square test result for 2 categorical variables 
    if they are independent or not"""
    target_arr = df[target].values
    expected = np.array([np.sum(target_arr)/2]*2)
    observed = np.squeeze(df.groupby(factor).agg({target:'sum'}).values)
    chi_result = chisquare(observed, expected)
    print('Observed Frequency: ' + str(observed))
    print('Expected Frequency: ' + str(expected))
    print('Chi2 Statistic: ' + str(int(chi_result[0])))
    print('P Value: ' + str(round(chi_result[1],4)))
    if chi_result[1]<alpha:
        print('Conclusion: 2 variables are dependent')
    else: 
        print('Conclusion: 2 variables are independent')

In [ ]:
print('Algorithm and Remote')
print('==================')
chisquaretest(df[df['group']!='a1'], 'alg', 'sng_use', 0.01)
print('')
print('Threshold and Remote')
print('==================')
chisquaretest(df[df['group']!='a1'], 'threshold', 'sng_use', 0.01)